In [1]:
from transformers import (pipeline, BertTokenizer, BertForMaskedLM, BertConfig, DataCollatorForLanguageModeling,
                            Trainer, TrainingArguments)
import torch
from datasets import load_dataset, load_from_disk, Dataset, DatasetDict, concatenate_datasets


In [2]:

unmasker = pipeline('fill-mask', model='bert-base-uncased')

unmasker("Hello I'm a [MASK] model.")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.10731099545955658,
  'token': 4827,
  'token_str': 'fashion',
  'sequence': "hello i'm a fashion model."},
 {'score': 0.08774477988481522,
  'token': 2535,
  'token_str': 'role',
  'sequence': "hello i'm a role model."},
 {'score': 0.05338403955101967,
  'token': 2047,
  'token_str': 'new',
  'sequence': "hello i'm a new model."},
 {'score': 0.04667219892144203,
  'token': 3565,
  'token_str': 'super',
  'sequence': "hello i'm a super model."},
 {'score': 0.02709587849676609,
  'token': 2986,
  'token_str': 'fine',
  'sequence': "hello i'm a fine model."}]

In [19]:
config = BertConfig.from_pretrained('bert-base-uncased')
model_path = "best_model.pt"
state_dict = torch.load(model_path)
model = BertForMaskedLM(config)
model.load_state_dict(state_dict)
model.eval()

RuntimeError: Error(s) in loading state_dict for BertForMaskedLM:
	Missing key(s) in state_dict: "cls.predictions.bias", "cls.predictions.transform.dense.weight", "cls.predictions.transform.dense.bias", "cls.predictions.transform.LayerNorm.weight", "cls.predictions.transform.LayerNorm.bias", "cls.predictions.decoder.weight", "cls.predictions.decoder.bias". 
	Unexpected key(s) in state_dict: "classifier.weight", "classifier.bias", "bert.pooler.dense.weight", "bert.pooler.dense.bias". 

In [ ]:
text = "The quick brown [MASK] jumps over the lazy dog."
tokenized_text = text.split()
tokenized_text[3] = '[MASK]'
masked_text = ' '.join(tokenized_text)

input = tokenizer.encode(masked_text, return_tensors="pt")
mask_token_index = torch.where(input == tokenizer.mask_token_id)[1]

token_logits = model(input).logits
mask_token_logits = token_logits[0, mask_token_index, :]
top_5_tokens = torch.topk(mask_token_logits, 5, dim=1).indices[0].tolist()

for token in top_5_tokens:

    print(tokenizer.decode(token))




c a t
d o g
b e a r
m a n
# # i e


In [20]:
from transformers import BertTokenizer, BertForMaskedLM
from sklearn.metrics import accuracy_score
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the model's predictions
predicted_words = []

for sentence in sentences:
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Use the model to predict the masked token
    outputs = model(**inputs)
    predictions = outputs.logits[0, mask_index, :]

    # Get the predicted token
    predicted_token = torch.argmax(predictions)
    predicted_word = tokenizer.decode([predicted_token])

    # Add the predicted word to the list
    predicted_words.append(predicted_word)

# Calculate the accuracy of the model's predictions
accuracy = accuracy_score(actual_words, predicted_words)

print(f"Accuracy: {accuracy}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Accuracy: 0.0


In [4]:
from transformers import BertTokenizer, BertForMaskedLM
import torch

# Load pre-trained model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

# Define sentences for benchmarking
sentences = ["Hello I'm a [MASK] model.", "The quick brown [MASK] jumps over the lazy dog."]

# Define the actual words that were masked
actual_words = ["language", "fox"]

# Initialize a list to store the losses
losses = []

for sentence, actual_word in zip(sentences, actual_words):
    # Encode the sentence and get the position of the masked token
    inputs = tokenizer.encode_plus(sentence, return_tensors='pt')
    mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

    # Replace the masked token with the actual token
    inputs["input_ids"][0, mask_index] = tokenizer.encode(actual_word, add_special_tokens=False)[0]

    # Use the model to calculate the loss
    outputs = model(**inputs, labels=inputs["input_ids"])
    loss = outputs.loss

    # Add the loss to the list
    losses.append(loss.item())

# Calculate the average loss
average_loss = sum(losses) / len(losses)

print(f"Average Loss: {average_loss}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Loss: 3.7346343994140625


In [5]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')

In [6]:
wiki_dataset

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [7]:
wiki_dataset["train"][4]["text"]

" The game began development in 2010 , carrying over a large portion of the work done on Valkyria Chronicles II . While it retained the standard features of the series , it also underwent multiple adjustments , such as making the game more forgiving for series newcomers . Character designer Raita Honjou and composer Hitoshi Sakimoto both returned from previous entries , along with Valkyria Chronicles II director Takeshi Ozawa . A large team of writers handled the script . The game 's opening theme was sung by May 'n . \n"

In [8]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [9]:
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


In [2]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

class TrainerWithCustomLoss(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.CrossEntropyLoss()
        loss = loss_fct(logits.view(-1, self.model.config.vocab_size), labels.view(-1))

        # if loss is nan, there is an issue with the labels, we return 0
        if torch.isnan(loss):
            loss = torch.tensor(0.0, requires_grad=True).to(loss.device)
        return (loss, outputs) if return_outputs else loss
    """
    def compute_loss(self, model, inputs, return_outputs=False):
        if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
        else:
            labels = None
        outputs = model(**inputs)

        # code for calculating accuracy
        if "labels" in inputs:
            logits = outputs.logits.detach()
            preds = logits.argmax(dim=1)
            acc1 = metric.compute(predictions=preds, references=inputs.labels)
            self.log({'accuracy_score': acc1})
            acc = (
                (preds.argmax(axis=-1) == inputs.labels.reshape(1, len(inputs.labels))[0])
                .type(torch.float)
                .mean()
                .item()
            )
            self.log({"train_accuracy": acc})
        # end code for calculating accuracy
                    
        # Save past state if it exists
        # TODO: this needs to be fixed and made cleaner later.
        if self.args.past_index >= 0:
            self._past = outputs[self.args.past_index]

        if labels is not None:
            loss = self.label_smoother(outputs, labels)
        else:
            # We don't use .loss here since the model may return tuples instead of ModelOutput.
            loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]

        return (loss, outputs) if return_outputs else loss
    """


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator
)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

NameError: name 'model' is not defined

In [23]:
tokenizer = BertTokenizer.from_pretrained('distilbert/distilbert-base-uncased')
model = BertForMaskedLM.from_pretrained('distilbert/distilbert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)
def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.
You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', '

In [24]:
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    prediction_loss_only=True,
    #label_names=[]
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return {"accuracy": (predictions == labels).mean()}

trainer = Trainer(
    model=model,
    args=training_args,
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator,
    compute_metrics=compute_metrics

)

# get loss on test set
#trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])

{'eval_loss': 10.404870986938477,
 'eval_runtime': 210.8167,
 'eval_samples_per_second': 20.672,
 'eval_steps_per_second': 1.295}

# Try to make BERT degenerate

In [26]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


# training with high lr
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-2,
    num_train_epochs=1
)


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    train_dataset=wiki_dataset_tokenized["train"].select(range(100)),
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator

)

# get loss on test set
trainer.train()
trainer.evaluate(wiki_dataset_tokenized["test"])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss


KeyboardInterrupt: 

In [5]:

unmasker = pipeline('fill-mask', model=model, tokenizer=tokenizer)

preds = unmasker("Hello I'm a [MASK] model.")

# Without Trainer

In [22]:
import numpy as np
test_text = wiki_dataset["test"]["text"]

np.random.seed(42)

# discard empty text and with less than 100 characters
test_text = [t for t in test_text if len(t) > 100]

# sample 100 texts
test_text = np.random.choice(test_text, 100)

# mask 1 word in each text
mask_indices = [np.random.randint(0, len(t.split())) for t in test_text]


# mask the tokens
masked_text = []
for i, t in enumerate(test_text):
    tokenized_text = t.split()
    tokenized_text[mask_indices[i]] = '[MASK]'
    masked_text.append(' '.join(tokenized_text))

for i in range(10):
    print("Text: ")
    print(masked_text[i])

Text: 
Writing in Variety , Todd McCarthy said the cast ensemble " could not be better " ; he praised Spacey 's " handling of innuendo , subtle sarcasm , and blunt talk " and the way he imbued Lester with " genuine feeling " . Janet Maslin in The New York Times said Spacey was at his " wittiest and most agile " to date , and Roger Ebert of the Chicago Sun @-@ Times singled Spacey out for successfully portraying a man who " does reckless and foolish things [ but who ] doesn 't deceive himself " . Kevin Jackson of Sight & Sound said Spacey impressed in ways distinct from his previous performances , the most satisfying aspect being his portrayal of " both sap and hero " . Writing in Film Quarterly , Gary Hentzi praised the actors , but said that characters such as Carolyn and Col. Fitts were stereotypes . Hentzi accused Mendes and Ball of identifying too readily with Jane and Ricky , saying the latter was their " fantasy figure " — a teenaged boy who 's an absurdly wealthy artist able to 

In [23]:
# compute loss with reference model on sampled texts
losses = []

reference_model = BertForMaskedLM.from_pretrained('bert-base-uncased')
reference_model.eval()
with torch.no_grad():
    for i, t in enumerate(masked_text):
        inputs = tokenizer.encode_plus(t, return_tensors='pt')
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        outputs = reference_model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")


Average Loss: 8.05634437084198


In [25]:
# compute loss with reference model on sampled texts
losses = []

# use distilbert as a placeholder for the finetuned_model
finetuned_model = BertForMaskedLM.from_pretrained('distilbert-base-uncased')
finetuned_model.eval()
with torch.no_grad():
    for i, t in enumerate(masked_text):
        inputs = tokenizer.encode_plus(t, return_tensors='pt')
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        outputs = finetuned_model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForMaskedLM were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attentio

Average Loss: 10.472420358657837


In [28]:
wiki_dataset = load_dataset('wikitext', 'wikitext-2-raw-v1')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained('bert-base-uncased')

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

def tokenize_text(x, tokenizer):
    return tokenizer(x["text"], truncation=True, padding="max_length", return_tensors="pt", max_length=200)

wiki_dataset_tokenized = wiki_dataset.map( lambda x: tokenize_text(x, tokenizer), batched=True)


# training with high lr
training_args = TrainingArguments(
    output_dir="./bert",
    overwrite_output_dir=True,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=5e-2,
    num_train_epochs=1
)


trainer = TrainerWithCustomLoss(
    model=model,
    args=training_args,
    train_dataset=wiki_dataset_tokenized["train"].select(range(100)),
    eval_dataset=wiki_dataset_tokenized["test"],
    data_collator=data_collator

)

# compute loss with reference model on sampled texts
losses = []

# use distilbert as a placeholder for the finetuned_model
finetuned_model = model
finetuned_model.eval()
with torch.no_grad():
    for i, t in enumerate(masked_text):
        inputs = tokenizer.encode_plus(t, return_tensors='pt').to(finetuned_model.device)
        mask_index = torch.where(inputs["input_ids"][0] == tokenizer.mask_token_id)

        outputs = finetuned_model(**inputs, labels=inputs["input_ids"])
        loss = outputs.loss
        losses.append(loss.item())

print(f"Average Loss: {sum(losses) / len(losses)}")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Average Loss: 0.6526461672782898


# Testing JudgeLM